In [419]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [420]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
184,tt2209764,100000000,103039258,Transcendence,Johnny Depp|Paul Bettany|Rebecca Hall|Kate Mar...,Wally Pfister,"Yesterday, Dr. Will Caster was only human...",Two leading computer scientists work toward th...,119,Thriller|Science Fiction|Drama|Mystery,Alcon Entertainment|Syncopy|DMG Entertainment|...,4/16/2014,5.9,2014
512,tt0133152,100000000,362211740,Planet of the Apes,Mark Wahlberg|Tim Roth|Helena Bonham Carter|Mi...,Tim Burton,You'll be sorry you were ever born human,After a spectacular crash-landing on an unchar...,119,Thriller|Science Fiction|Action|Adventure,Twentieth Century Fox Film Corporation|Tim Bur...,7/25/2001,5.5,2001
1004,tt1397514,79000000,355692760,Journey 2: The Mysterious Island,Dwayne Johnson|Josh Hutcherson|Kristin Davis|V...,Brad Peyton,Believe the Impossible. Discover the Incredible.,Sean Anderson partners with his mom's boyfrien...,94,Adventure|Action|Science Fiction,New Line Cinema|Contrafilm|Walden Media,1/19/2012,5.8,2012
1395,tt0384642,45000000,56070433,Kicking & Screaming,Will Ferrell|Robert Duvall|Kate Walsh|Musetta ...,Jesse Dylan,One man could lead this team to glory. . . Tha...,Phil Weston has been unathletic his entire lif...,95,Romance|Comedy|Family,Universal Pictures,5/13/2005,5.8,2005
1299,tt1487931,20000000,27187375,Khumba,Jake T. Austin|Liam Neeson|Steve Buscemi|AnnaS...,Anthony Silverston,"Half a zebra, a whole lot of adventure!",A half-striped zebra is blamed for the drought...,85,Animation|Adventure|Family,Triggerfish Animation,7/18/2013,5.8,2013


In [400]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [401]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [325]:
answers = {} # создадим словарь для ответов

In [421]:
# тут другие ваши предобработки колонок например:

# Дата release_date имеет тип string (object).
# Чтобы работать с ней как с датой, нужно поменять тип на datetime
from datetime import datetime

data['release_date'] = data.release_date.apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))

# Для удобства группировки добавляем месяц релиза
data['release_month'] = data.release_date.apply(lambda x: x.month)

# Также для удобства группировки делаем фунциию, определяющую сезон
def season(month):
    if month in [1,2,12]:
        return 'winter'
    if month in [3,4,5]:
        return 'spring'
    if month in [6,7,8]:
        return 'summer'
    return 'autumn'
    
# и добавляем сезон в датасет
data['release_season'] = data.release_month.apply(season)

# добавляем расчетную колонку profit = revenue - budget
data['profit'] = data.revenue - data.budget

# для №27 - функиця, которая парсит имена актеров и возвращает их попарные сочетания
def split_into_pairs(cast):
    ''' Функиця принимает строку с именами актеров, разделенными '|', и возвращает их попарные сочетания в виде списка кортежей.
        
        Важно: предусмотреть, чтобы не формировались пары вида (A B) и (B A), которые при подсчете будут 
            восприниматься как разные.
        
        Для этого:
        1. Сортируем список с именами, чтобы убрать различия в порядке перечисления актеров в разных фильмах,
        2. Сочетания строим внутри двойного цикла, где внешний перебирает имена в списке, а внутренний перебирает имена 
             с меньшим индексом в этом же списке,
        3. Сочетания имен с совпадающими индексами не создаем.
        
        Кортеж нужен для корректной работы value_counts(), т.к. когда value_counts() применяется к значениям типа list,
            интерпретатор выдет предупреждение, хотя и возвращает корректный результат.
    ''' 
    split_cast = []
    split_cast = cast.split('|')
    split_cast.sort()       # сотрировка нужна, чтобы обеспечить единый порядок имен актеров среди всех фильмов 
    cast_pairs = []
    i=0

    for i in range (0, len(split_cast)):                 # перебираем имена в списке
        for j in range(0,i+1):                           # перебираем имена с меньшим индексом в том же списке
            if i!=j:                                     # пары с совпадающим индексом пропускаем
                cast_pair = tuple([split_cast[j],split_cast[i]])  
                cast_pairs.append(cast_pair)                     # собираем полученные пары в список
                
    return cast_pairs                                           # возвращаем список кортежей с парами актеров

# используем функцю, чтобы добавить в датасет списки с парами актеров
data['cast_pairs'] = data.cast.apply(split_into_pairs)


In [403]:
data.describe()

,budget,revenue,runtime,vote_average,release_year,release_month,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,6.933298,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,3.390196,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,1.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,4.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,7.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,10.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,12.000000,2.544506e+09


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [71]:
# выбираем фильм с максимальным бюжетом
data[data['budget']==max(data['budget'])][['original_title','imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


In [83]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [70]:
# выбираем фильм с максимальным значением runtime
data[data['runtime']==max(data['runtime'])][['original_title','imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


In [327]:
answers['2'] = 'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [69]:
# выбираем фильм с минимальным значением runtime
data[data['runtime']==min(data['runtime'])][['original_title','imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


In [328]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [68]:
# просто считаем среднее значение runtime по всем записям датасета
data['runtime'].mean()

109.6585494970884

In [329]:
answers['4'] = '110'

# 5. Каково медианное значение длительности фильмов? 

In [67]:
# просто считаем медианное значение runtime по всем записям датасета
data['runtime'].median()

107.0

In [330]:
answers['5'] = '107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [66]:
# выбираем фильм с максимальным значением profit
data[data['profit']==max(data['profit'])][['original_title','imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


In [331]:
answers['6'] = 'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [65]:
# выбираем фильм с минимальным значением profit
data[data['profit']==min(data['profit'])][['original_title','imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


In [332]:
answers['7'] = 'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [64]:
# считаем фильмы, где profit превысил бюджет
data.query('profit>0').imdb_id.count()

1478

In [333]:
answers['8'] = '1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [92]:
# сначала выберем все фильмы, вышедшие в 2008 г., затем из полученнго сабсета выберем фильм с максимальным revenue
d1 = data.query('release_year==2008')
d1[d1['revenue']==max(d1['revenue'])][['original_title','imdb_id']]

,original_title,imdb_id
599,The Dark Knight,tt0468569


In [334]:
answers['9'] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [98]:
# сначала выберем все фильмы, вышедшие с 2012 по 2014 г., затем из полученнго сабсета выберем фильм с минимальным профитом
d1 = data.query('2012<=release_year<=2014')
d1[d1['profit']==min(d1['profit'])][['original_title','imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


In [335]:
answers['10'] = 'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [126]:
# для экономии памяти для работы выбираем один столбец genres, парсим из него названия отдельных жанров
# применяем функцию pandas explode(), чтобы растащить жанры из полученных списков в отдельные записи 
# получившиеся записи считаем через value_counts()
d1 = data['genres']
d1 = d1.apply(lambda s: s.split('|')).explode()
display(d1.value_counts().head(1))

Drama    782
Name: genres, dtype: int64

In [336]:
answers['11'] = 'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [125]:
# поступаем точно так же, как в предыдущей задаче, с той разницей, что для работы выбираем только прибыльные фильмы
d1 = data.query('profit>0')['genres']
d1 = d1.apply(lambda s: s.split('|')).explode()
display(d1.value_counts().head(1))

Drama    560
Name: genres, dtype: int64

In [337]:
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [199]:
# выбираем столбцы, с которыми будем работать - director и revenue
d1 = data[['director', 'revenue']]

# здесь у меня не получилось распарсить имена режиссеров внутри столбца - интерпретатор выдавал ошибку
# поэтому сделал через concat 
d2 = pd.concat([d1['director'].apply(lambda s: s.split('|')),d1['revenue']],axis=1)

# выделяем отдельные имена режиссеров с помощью explode()
d2 = d2.explode('director')

# считаем сборы с группировкой по режиссеру
d3 = d2.groupby(['director'])['revenue'].sum().reset_index()   # reset_index() для корректной сортировки

# выбираем запись с максимальным значением revenue
d3.sort_values('revenue',ascending=False).head(1)

,director,revenue
727,Peter Jackson,6490593685


In [338]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [203]:
# для работы выбираем столбец с режиссерами, в жанре фильмов которых есть "Action"
d1 = data[data.genres.str.contains("Action", na=False)]['director']

# парсим имена режиссеров и разделяем на отдельные имена через explode()
d1 = d1.apply(lambda s: s.split('|')).explode()

# находим самое частое имя
d1.value_counts().head(1)

Robert Rodriguez    9
Name: director, dtype: int64

In [339]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [212]:
# сначала выбираем столбцы cast и revenue для фильмов, вышедших в 2012 г
d1 = data.query('release_year==2012')[['cast', 'revenue']]

# как и в задаче ранее, преобразую cast в список, через concat снова присоединяю столбец revenue
d2 = pd.concat([d1['cast'].apply(lambda s: s.split('|')),d1['revenue']],axis=1)

# разделяем на отдельные имена актеров
d2 = d2.explode('cast')

# суммируем revenue с группировкой по актерам
d3 = d2.groupby(['cast'])['revenue'].sum().reset_index()

# выбираем актера, для которого revenue был максимальным
d3.sort_values('revenue',ascending=False).head(1)

,cast,revenue
77,Chris Hemsworth,2027450773


In [340]:
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [238]:
# определяем высокобюджетный фильм как фильм, чей бюджет был выше среднего
# для критерия считаем среднее значение бюджета
mean_budget = data.budget.mean()

# выбираем актеров, снявшихся в фильмах, чей бюджет выше среднего значения
d1 = data[data['budget']>mean_budget]['cast']

# парсим отдельные имена
d1 = d1.apply(lambda s: s.split('|')).explode()

# находим самое частое имя
d1.value_counts().head(1)


Matt Damon    18
Name: cast, dtype: int64

In [341]:
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [221]:
# выбираем жанры фильмов, в актерском составе которых был Nicolas Cage
d1 = data[data.cast.str.contains("Nicolas Cage", na=False)]['genres']

# парсим отдельные названия жанров
d1 = d1.apply(lambda s: s.split('|')).explode()

# находим самый частый жанр
d1.value_counts().head(1)

Action    17
Name: genres, dtype: int64

In [342]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [252]:
# выбираем фильмы и профит по фильмам, в названии студий которых есть "Paramount"
d1 = data[data.production_companies.str.contains("Paramount", na=False)][['original_title','imdb_id','profit']]

# выбираем фильм с минимальным значением профита
d1[d1['profit']==min(d1['profit'])][['original_title','imdb_id']]

,original_title,imdb_id
925,K-19: The Widowmaker,tt0267626


In [343]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [257]:
# для работы выюираем год выпуска и выручку
d1 = data[['release_year', 'revenue']]

# суммируем выручку с группировкой по годам
d2 = d1.groupby(['release_year'])['revenue'].sum().reset_index()

# сортируем года по сумме выручки, берем верхний в списке
d2.sort_values('revenue',ascending=False).head(1)

,release_year,revenue
15,2015,25449202382


In [344]:
answers['19'] = '2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [262]:
# аналогично предыдущей задаче выбираем год и профит по фильмам, в названии студии которых есть "Warner"
# для критерия используем частичное имя студии, т.к. в датасете есть несколько ее подразделений,
# которые тоже относятся к этой студии
d1 = data[data.production_companies.str.contains("Warner", na=False)][['release_year','profit']]

# суммируем профит по годам
d2 = d1.groupby(['release_year'])['profit'].sum().reset_index()

# сортируем года по сумме профита, берем верхний в списке
d2.sort_values('profit',ascending=False).head(1)

,release_year,profit
14,2014,2295464519


In [345]:
answers['20'] = '2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [270]:
# с помощью вспомогательного поля release_month (см. Предобработку) легко считаем самый частый месяц в датасете
data.release_month.value_counts().head(1)

9    227
Name: release_month, dtype: int64

In [346]:
answers['21'] = '9'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [271]:
# с помощью вспомогательного поля release_season (см. Предобработку) легко считаем количество записей для лета
data[data.release_season=='summer']['imdb_id'].count()

450

In [347]:
answers['22'] = '450'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [276]:
# выбираем столбец director, фильтруя датасет по нужному значению сезона (см. Предобработку) 
d1 = data[data.release_season=='winter']['director']

# парсим имена режиссеров
d1 = d1.apply(lambda s: s.split('|')).explode()

# ищем самое частое имя
d1.value_counts().head(1)

Peter Jackson    7
Name: director, dtype: int64

In [348]:
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [288]:
# выбираем названия студий и фильмов
d1 = data[['production_companies', 'original_title']]

# преобразуем названия студий в списки 
d2 = pd.concat([d1['production_companies'].apply(lambda s: s.split('|')),d1['original_title']],axis=1)

# парсим отдельные названия студий
d2 = d2.explode('production_companies')

# к датафрейму добавляем столбец, где считаем длину названия фильма
d2['title_length'] = d2.original_title.str.len()

# считаем среднюю длину названия фильма по каждой студии с помощью группировки
d3 = d2.groupby(['production_companies'])['title_length'].mean().reset_index()

# сортируем названия студий по убыванию значения средней длины названия фильма, берем верхнюю в списке
d3.sort_values('title_length',ascending=False).head(1)

,production_companies,title_length
583,Four By Two Productions,83.0


In [349]:
answers['24'] = 'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [312]:
# выбираем столбцы с названиями студий и описаниями фильмов
d1 = data[['production_companies', 'overview']]

# преобразуем названия студий в списки 
d2 = pd.concat([d1['production_companies'].apply(lambda s: s.split('|')),d1['overview']],axis=1)

# парсим отдельные названия студий
d2 = d2.explode('production_companies')

# к датафрейму добавляем столбец, где разбиваем описания фильмов на слова
d2['overview_words'] = d2.overview.str.split( )

# добавляем еще один столбец, где считаем количество слов в каждом описании
d2['overview_words_count'] = d2.overview_words.str.len()

# считаем среднее количество слов в описании фильма по каждой студии с помощью группировки
d3 = d2.groupby(['production_companies'])['overview_words_count'].mean().reset_index()

# сортируем названия студий по убыванию значения среднего количества слов в описании фильма, берем верхнюю в списке
d3.sort_values('overview_words_count',ascending=False).head(1)

,production_companies,overview_words_count
1041,Midnight Picture Show,175.0


In [350]:
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [451]:
# выбираем названия фильмов и значения рейтинга
d1 = data[['original_title','vote_average']]

# выбираем фильмы, чей рейтинг больше рейтинга остальных 99% фильмов с помощью функции quantile()
d2 = d1[d1['vote_average'] > d1.quantile(.99)[0]].sort_values(['vote_average'], ascending=False)
d2

,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
125,The Imitation Game,8.0
119,Guardians of the Galaxy,7.9
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
872,The Pianist,7.9


In [372]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [425]:
# используем вспомогательный столбец cast_pairs, полученный с помощью функции split_into_pairs (см. Предобработку)
# столбец содержит списки с кортежами отсортированных пар имен актеров, которые теперь можно распарсить с помощью explode()
d1 = data['cast_pairs'].explode()

# находим самую частую пару
d1.value_counts().head(10)

(Daniel Radcliffe, Rupert Grint)       8
(Daniel Radcliffe, Emma Watson)        8
(Emma Watson, Rupert Grint)            8
(Helena Bonham Carter, Johnny Depp)    6
(Ben Stiller, Owen Wilson)             6
(Robert Pattinson, Taylor Lautner)     5
(Adam Sandler, Kevin James)            5
(Kristen Stewart, Robert Pattinson)    5
(Paul Walker, Vin Diesel)              5
(Hugh Jackman, Ian McKellen)           5
Name: cast_pairs, dtype: int64

In [426]:
answers['27'] = 'Daniel Radcliffe, Rupert Grint'

ВАРИАНТ 2

# Submission

In [427]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': '9',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe, Rupert Grint'}

In [428]:
# и убедиться что ни чего не пропустил)
len(answers)

27